### Task 2: Determine within which grid cell a point/particle falls into
### Task 3: Interpolate velocity to the particle lat/lon 
### Task 4: Advance forward

- variables u,v (current velocity components)
- use only surface data: depth (z) = -2.0 
- use first time for now [0]

Note: current velocities u, v are on different grids, u(eta_u,xi_u) and v(eta_v, xi_v).
Latitudes and longitudes corresponding values for current are in the coordinate arrayes lat_u, lon_u, lat_v, lon_v


In [26]:
# task1: acquire data
import numpy as np
import pandas as pd
import xarray as xr

url='https://oceanmodeling.ucsc.edu:8443/thredds/dodsC/ccsra_2016a_phys_agg_zlevs/fmrc/CCSRA_2016a_Phys_ROMS_z-level_(depth)_Aggregation_best.ncd'
actual = xr.open_dataset(url)

# ds = actual.copy()              # ("ds.nc")


lat1 = 38.0
lon1 = -123.0

In [30]:
# end_index = 15
end_index = len(actual.time.values)
master = actual.sel(time = actual.time.values[0])
for i in range(1, end_index):
    temp = actual.sel(
        time = actual.time.values[i]
    )
    master = xr.concat([master, temp], dim='time')
    print(i)
master.to_netcdf("ds.nc")
t = xr.open_dataset("ds.nc")
t

1
2
3
4
5
6
7
8
9
10
11
12
13
14


<xarray.Dataset>
Dimensions:      (eta_rho: 181, xi_rho: 186, eta_u: 181, xi_u: 185, eta_v: 180, xi_v: 186, ocean_time: 1, z: 11, time: 15)
Coordinates:
    lat_rho      (eta_rho, xi_rho) float64 ...
    lat_u        (eta_u, xi_u) float64 ...
    lat_v        (eta_v, xi_v) float64 ...
    lon_rho      (eta_rho, xi_rho) float64 ...
    lon_u        (eta_u, xi_u) float64 ...
    lon_v        (eta_v, xi_v) float64 ...
  * ocean_time   (ocean_time) datetime64[ns] 2022-07-09
  * z            (z) float64 -250.0 -200.0 -150.0 -100.0 ... -10.0 -5.0 -2.0
  * time         (time) datetime64[ns] 2011-01-02 2011-01-03 ... 2011-01-16
    time_run     (time) datetime64[ns] ...
Dimensions without coordinates: eta_rho, xi_rho, eta_u, xi_u, eta_v, xi_v
Data variables: (12/15)
    angle        (time, eta_rho, xi_rho) float64 ...
    h            (time, eta_rho, xi_rho) float64 ...
    mask_rho     (time, eta_rho, xi_rho) float64 ...
    mask_u       (time, eta_u, xi_u) float64 ...
    mask_v       (time, eta_v, xi_v) float64 ...
    ubar         (time, eta_u, xi_u) float64 ...
    ...           ...
    salt         (time, z, eta_rho, xi_rho) float32 ...
    rho          (time, z, eta_rho, xi_rho) float32 ...
    u            (time, z, eta_u, xi_u) float32 ...
    v            (time, z, eta_v, xi_v) float32 ...
    w            (time, z, eta_rho, xi_rho) float32 ...
    time_offset  (time) datetime64[ns] ...
Attributes: (12/46)
    file:                            output/da_fwd_002.nc
    format:                          netCDF-3 64bit offset file
    Conventions:                     CF-1.4
    type:                            ROMS/TOMS history file
    title:                           UCSC California Current System, wc12.0_c...
    state_vector:                    zeta, ubar, vbar, u, v, temp, salt
    ...                              ...
    NCO:                             "4.5.2"
    _CoordSysBuilder:                ucar.nc2.dataset.conv.CF1Convention
    cdm_data_type:                   GRID
    featureType:                     GRID
    location:                        Proto fmrc:CCSRA_2016a_Phys_ROMS_z-level...
    DODS_EXTRA.Unlimited_Dimension:  ocean_time

In [39]:
# Task2: find nearest grid points for current components u and v.

for x in range(len(ds.lon_u)):
    print(ds.lon_u[0, x].lon_u.values)

-133.95
-133.85000000000002
-133.75
-133.64999999999998
-133.55
-133.45
-133.35000000000002
-133.25
-133.14999999999998
-133.05
-132.95
-132.85000000000002
-132.75
-132.64999999999998
-132.55
-132.45
-132.35000000000002
-132.25
-132.14999999999998
-132.05
-131.95
-131.85000000000002
-131.75
-131.64999999999998
-131.55
-131.45
-131.35000000000002
-131.25
-131.14999999999998
-131.05
-130.95
-130.85000000000002
-130.75
-130.64999999999998
-130.55
-130.45
-130.35000000000002
-130.25
-130.14999999999998
-130.05
-129.95
-129.85000000000002
-129.75
-129.64999999999998
-129.55
-129.45
-129.35000000000002
-129.25
-129.14999999999998
-129.05
-128.95
-128.85000000000002
-128.75
-128.64999999999998
-128.55
-128.45
-128.35000000000002
-128.25
-128.14999999999998
-128.05
-127.95
-127.85
-127.75
-127.65
-127.55
-127.45
-127.35
-127.25
-127.15
-127.05
-126.95
-126.85
-126.75
-126.65
-126.55
-126.45
-126.35
-126.25
-126.15
-126.05
-125.95
-125.85
-125.75
-125.65
-125.55
-125.45
-125.35
-125.25
-125.15


In [41]:
# Task3: interpolate velocity (u,v) to lat1 and lat2 (this point might not fall on a grid point)

ds.interp(eta_u=ds.eta_rho, xi_u=ds.xi_rho, method='linear')

RuntimeError: NetCDF: Authorization failure

In [ ]:
# Task4: move particle forward 
# we need to calculate the position of the particle if it moves for one time step (a day in this data set) with velocity u, v
# there are different methods to implement that, we will use the Runge-Kutta order 4. but you can start w an smipl